In [1]:
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_regression

from hyperopt import hp
from hyperopt import Trials

from xgboost import *

try:
    from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA
except:
    !pip install --upgrade shap-hypetune
    from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA

import warnings
warnings.simplefilter('ignore')

In [2]:
X_clf, y_clf = make_classification(n_samples=6000, n_features=20, n_classes=2, 
                                   n_informative=4, n_redundant=6, random_state=0)

X_clf_train, X_clf_valid, y_clf_train, y_clf_valid = train_test_split(
    X_clf, y_clf, test_size=0.3, shuffle=False)

X_regr, y_regr = make_classification(n_samples=6000, n_features=20,
                                     n_informative=7, random_state=0)

X_regr_train, X_regr_valid, y_regr_train, y_regr_valid = train_test_split(
    X_regr, y_regr, test_size=0.3, shuffle=False)

In [3]:
param_grid = {
    'learning_rate': [0.2, 0.1],
    'num_leaves': [25, 35],
    'max_depth': [10, 12]
}

param_dist = {
    'learning_rate': stats.uniform(0.09, 0.25),
    'num_leaves': stats.randint(20,40),
    'max_depth': [10, 12]
}

param_dist_hyperopt = {
    'max_depth': 15 + hp.randint('num_leaves', 5), 
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}


regr_xgb = XGBRegressor(n_estimators=150, random_state=0, verbosity=0, n_jobs=-1)
clf_xgb = XGBClassifier(n_estimators=150, random_state=0, verbosity=0, n_jobs=-1)

# Hyperparameters Tuning

In [4]:
### HYPERPARAM TUNING WITH GRID-SEARCH ###

model = BoostSearch(clf_xgb, param_grid=param_grid)
model.fit(X_clf_train, y_clf_train, eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0)


8 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00021 ### eval_score: 0.2045
trial: 0002 ### iterations: 00026 ### eval_score: 0.19472
trial: 0003 ### iterations: 00021 ### eval_score: 0.2045
trial: 0004 ### iterations: 00026 ### eval_score: 0.19472
trial: 0005 ### iterations: 00045 ### eval_score: 0.19964
trial: 0006 ### iterations: 00050 ### eval_score: 0.20157
trial: 0007 ### iterations: 00045 ### eval_score: 0.19964
trial: 0008 ### iterations: 00050 ### eval_score: 0.20157


BoostSearch(estimator=XGBClassifier(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=150, n_jobs=-1,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=0, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                               

In [5]:
model.estimator_, model.best_params_, model.best_score_

(XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.2, max_delta_step=0,
               max_depth=12, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=150, n_jobs=-1,
               num_leaves=25, num_parallel_tree=1, predictor='auto',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
               subsample=1, tree_method='exact', validate_parameters=1,
               verbosity=0),
 {'learning_rate': 0.2, 'num_leaves': 25, 'max_depth': 12},
 0.194719)

In [6]:
(model.score(X_clf_valid, y_clf_valid), 
 model.predict(X_clf_valid).shape, 
 model.predict_proba(X_clf_valid).shape)

(0.9138888888888889, (1800,), (1800, 2))

In [7]:
### HYPERPARAM TUNING WITH RANDOM-SEARCH ###

model = BoostSearch(
    regr_xgb, param_grid=param_dist,
    n_iter=8, sampling_seed=0
)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)


8 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00012 ### eval_score: 0.27616
trial: 0002 ### iterations: 00056 ### eval_score: 0.26211
trial: 0003 ### iterations: 00078 ### eval_score: 0.27603
trial: 0004 ### iterations: 00045 ### eval_score: 0.26117
trial: 0005 ### iterations: 00046 ### eval_score: 0.27868
trial: 0006 ### iterations: 00035 ### eval_score: 0.27815
trial: 0007 ### iterations: 00039 ### eval_score: 0.2753
trial: 0008 ### iterations: 00016 ### eval_score: 0.28116


BoostSearch(estimator=XGBRegressor(base_score=None, booster=None,
                                   colsample_bylevel=None,
                                   colsample_bynode=None, colsample_bytree=None,
                                   enable_categorical=False, gamma=None,
                                   gpu_id=None, importance_type=None,
                                   interaction_constraints=None,
                                   learning_rate=None, max_delta_step=None,
                                   max_depth=None, min_child_weight=None,
                                   missing=nan, monotone_constraints=None,
                                   n_estim...
                                   random_state=0, reg_alpha=None,
                                   reg_lambda=None, scale_pos_weight=None,
                                   subsample=None, tree_method=None,
                                   validate_parameters=None, verbosity=0),
            n_iter=8,
       

In [8]:
model.estimator_, model.best_params_, model.best_score_

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1669837381562427,
              max_delta_step=0, max_depth=10, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=150, n_jobs=-1,
              num_leaves=25, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=0),
 {'learning_rate': 0.1669837381562427, 'num_leaves': 25, 'max_depth': 10},
 0.26117)

In [9]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape)

(0.7271524639165458, (1800,))

In [10]:
### HYPERPARAM TUNING WITH HYPEROPT ###

model = BoostSearch(
    regr_xgb, param_grid=param_dist_hyperopt,
    n_iter=8, sampling_seed=0
)
model.fit(
    X_regr_train, y_regr_train, trials=Trials(), 
    eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0
)


8 trials detected for ('max_depth', 'learning_rate', 'colsample_bytree')

trial: 0001 ### iterations: 00149 ### eval_score: 0.27498
trial: 0002 ### iterations: 00074 ### eval_score: 0.27186
trial: 0003 ### iterations: 00038 ### eval_score: 0.28326
trial: 0004 ### iterations: 00149 ### eval_score: 0.29455
trial: 0005 ### iterations: 00149 ### eval_score: 0.28037
trial: 0006 ### iterations: 00149 ### eval_score: 0.26421
trial: 0007 ### iterations: 00052 ### eval_score: 0.27191
trial: 0008 ### iterations: 00133 ### eval_score: 0.29251


BoostSearch(estimator=XGBRegressor(base_score=None, booster=None,
                                   colsample_bylevel=None,
                                   colsample_bynode=None, colsample_bytree=None,
                                   enable_categorical=False, gamma=None,
                                   gpu_id=None, importance_type=None,
                                   interaction_constraints=None,
                                   learning_rate=None, max_delta_step=None,
                                   max_depth=None, min_child_weight=None,
                                   missing=nan, monotone_constraints=None,
                                   n_estim...
                                   random_state=0, reg_alpha=None,
                                   reg_lambda=None, scale_pos_weight=None,
                                   subsample=None, tree_method=None,
                                   validate_parameters=None, verbosity=0),
            n_iter=8,
       

In [11]:
model.estimator_, model.best_params_, model.best_score_

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7597292534356749,
              enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.059836658149176665,
              max_delta_step=0, max_depth=16, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=150, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=0),
 {'colsample_bytree': 0.7597292534356749,
  'learning_rate': 0.059836658149176665,
  'max_depth': 16},
 0.264211)

In [12]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape)

(0.7207605727361562, (1800,))

# Features Selection

In [13]:
### BORUTA ###

model = BoostBoruta(clf_xgb, max_iter=200, perc=100)
model.fit(X_clf_train, y_clf_train, eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0)

BoostBoruta(estimator=XGBClassifier(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=150, n_jobs=-1,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=0, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                               

In [14]:
model.estimator_, model.n_features_

(XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.300000012,
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=150, n_jobs=-1,
               num_parallel_tree=1, predictor='auto', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
               tree_method='exact', validate_parameters=1, verbosity=0),
 11)

In [15]:
(model.score(X_clf_valid, y_clf_valid), 
 model.predict(X_clf_valid).shape, 
 model.transform(X_clf_valid).shape,
 model.predict_proba(X_clf_valid).shape)

(0.9161111111111111, (1800,), (1800, 11), (1800, 2))

In [16]:
### RECURSIVE FEATURE ELIMINATION (RFE) ###

model = BoostRFE(regr_xgb, min_features_to_select=1, step=1)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)

BoostRFE(estimator=XGBRegressor(base_score=None, booster=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, enable_categorical=False,
                                gamma=None, gpu_id=None, importance_type=None,
                                interaction_constraints=None,
                                learning_rate=None, max_delta_step=None,
                                max_depth=None, min_child_weight=None,
                                missing=nan, monotone_constraints=None,
                                n_estimators=150, n_jobs=-1,
                                num_parallel_tree=None, predictor=None,
                                random_state=0, reg_alpha=None, reg_lambda=None,
                                scale_pos_weight=None, subsample=None,
                                tree_method=None, validate_parameters=None,
                                verbosity=0),
         min_

In [17]:
model.estimator_, model.n_features_

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=150, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=0),
 7)

In [18]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape)

(0.7317444492376407, (1800,), (1800, 7))

In [19]:
### RECURSIVE FEATURE ADDITION (RFA) ###

model = BoostRFA(regr_xgb, min_features_to_select=1, step=1)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)

BoostRFA(estimator=XGBRegressor(base_score=None, booster=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, enable_categorical=False,
                                gamma=None, gpu_id=None, importance_type=None,
                                interaction_constraints=None,
                                learning_rate=None, max_delta_step=None,
                                max_depth=None, min_child_weight=None,
                                missing=nan, monotone_constraints=None,
                                n_estimators=150, n_jobs=-1,
                                num_parallel_tree=None, predictor=None,
                                random_state=0, reg_alpha=None, reg_lambda=None,
                                scale_pos_weight=None, subsample=None,
                                tree_method=None, validate_parameters=None,
                                verbosity=0),
         min_

In [20]:
model.estimator_, model.n_features_

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=150, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=0),
 8)

In [21]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape)

(0.7274037362877257, (1800,), (1800, 8))

# Features Selection with SHAP

In [22]:
### BORUTA SHAP ###

model = BoostBoruta(
    clf_xgb, max_iter=200, perc=100,
    importance_type='shap_importances', train_importance=False
)
model.fit(X_clf_train, y_clf_train, eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0)

BoostBoruta(estimator=XGBClassifier(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=150, n_jobs=-1,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=0, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                               

In [23]:
model.estimator_, model.n_features_

(XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.300000012,
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=150, n_jobs=-1,
               num_parallel_tree=1, predictor='auto', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
               tree_method='exact', validate_parameters=1, verbosity=0),
 10)

In [24]:
(model.score(X_clf_valid, y_clf_valid), 
 model.predict(X_clf_valid).shape, 
 model.transform(X_clf_valid).shape,
 model.predict_proba(X_clf_valid).shape)

(0.91, (1800,), (1800, 10), (1800, 2))

In [25]:
### RECURSIVE FEATURE ELIMINATION (RFE) SHAP ###

model = BoostRFE(
    regr_xgb, min_features_to_select=1, step=1,
    importance_type='shap_importances', train_importance=False
)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)

BoostRFE(estimator=XGBRegressor(base_score=None, booster=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, enable_categorical=False,
                                gamma=None, gpu_id=None, importance_type=None,
                                interaction_constraints=None,
                                learning_rate=None, max_delta_step=None,
                                max_depth=None, min_child_weight=None,
                                missing=nan, monotone_constraints=None,
                                n_estimators=150, n_jobs=-1,
                                num_parallel_tree=None, predictor=None,
                                random_state=0, reg_alpha=None, reg_lambda=None,
                                scale_pos_weight=None, subsample=None,
                                tree_method=None, validate_parameters=None,
                                verbosity=0),
         impo

In [26]:
model.estimator_, model.n_features_

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=150, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=0),
 7)

In [27]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape)

(0.7317444492376407, (1800,), (1800, 7))

In [28]:
### RECURSIVE FEATURE ADDITION (RFA) SHAP ###

model = BoostRFA(
    regr_xgb, min_features_to_select=1, step=1,
    importance_type='shap_importances', train_importance=False
)
model.fit(
    X_regr_train, y_regr_train, trials=Trials(), 
    eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0
)

BoostRFA(estimator=XGBRegressor(base_score=None, booster=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, enable_categorical=False,
                                gamma=None, gpu_id=None, importance_type=None,
                                interaction_constraints=None,
                                learning_rate=None, max_delta_step=None,
                                max_depth=None, min_child_weight=None,
                                missing=nan, monotone_constraints=None,
                                n_estimators=150, n_jobs=-1,
                                num_parallel_tree=None, predictor=None,
                                random_state=0, reg_alpha=None, reg_lambda=None,
                                scale_pos_weight=None, subsample=None,
                                tree_method=None, validate_parameters=None,
                                verbosity=0),
         impo

In [29]:
model.estimator_, model.n_features_

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=150, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=0),
 9)

In [30]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape)

(0.7249664284333042, (1800,), (1800, 9))

# Hyperparameters Tuning + Features Selection

In [31]:
### HYPERPARAM TUNING WITH GRID-SEARCH + BORUTA ###

model = BoostBoruta(clf_xgb, param_grid=param_grid, max_iter=200, perc=100)
model.fit(X_clf_train, y_clf_train, eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0)


8 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00026 ### eval_score: 0.20001
trial: 0002 ### iterations: 00022 ### eval_score: 0.20348
trial: 0003 ### iterations: 00026 ### eval_score: 0.20001
trial: 0004 ### iterations: 00022 ### eval_score: 0.20348
trial: 0005 ### iterations: 00048 ### eval_score: 0.19925
trial: 0006 ### iterations: 00052 ### eval_score: 0.20307
trial: 0007 ### iterations: 00048 ### eval_score: 0.19925
trial: 0008 ### iterations: 00052 ### eval_score: 0.20307


BoostBoruta(estimator=XGBClassifier(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=150, n_jobs=-1,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=0, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                               

In [32]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

(XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.1, max_delta_step=0,
               max_depth=10, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=150, n_jobs=-1,
               num_leaves=25, num_parallel_tree=1, predictor='auto',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
               subsample=1, tree_method='exact', validate_parameters=1,
               verbosity=0),
 {'learning_rate': 0.1, 'num_leaves': 25, 'max_depth': 10},
 0.199248,
 11)

In [33]:
(model.score(X_clf_valid, y_clf_valid), 
 model.predict(X_clf_valid).shape, 
 model.transform(X_clf_valid).shape,
 model.predict_proba(X_clf_valid).shape)

(0.9144444444444444, (1800,), (1800, 11), (1800, 2))

In [34]:
### HYPERPARAM TUNING WITH RANDOM-SEARCH + RECURSIVE FEATURE ELIMINATION (RFE) ###

model = BoostRFE(
    regr_xgb, param_grid=param_dist, min_features_to_select=1, step=1,
    n_iter=8, sampling_seed=0
)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)


8 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00021 ### eval_score: 0.25941
trial: 0002 ### iterations: 00077 ### eval_score: 0.25055
trial: 0003 ### iterations: 00086 ### eval_score: 0.25676
trial: 0004 ### iterations: 00098 ### eval_score: 0.25383
trial: 0005 ### iterations: 00050 ### eval_score: 0.25751
trial: 0006 ### iterations: 00028 ### eval_score: 0.26007
trial: 0007 ### iterations: 00084 ### eval_score: 0.2603
trial: 0008 ### iterations: 00024 ### eval_score: 0.26278


BoostRFE(estimator=XGBRegressor(base_score=None, booster=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, enable_categorical=False,
                                gamma=None, gpu_id=None, importance_type=None,
                                interaction_constraints=None,
                                learning_rate=None, max_delta_step=None,
                                max_depth=None, min_child_weight=None,
                                missing=nan, monotone_constraints=None,
                                n_estimato...
                                random_state=0, reg_alpha=None, reg_lambda=None,
                                scale_pos_weight=None, subsample=None,
                                tree_method=None, validate_parameters=None,
                                verbosity=0),
         min_features_to_select=1, n_iter=8,
         param_grid={'learning_rate': <scipy.stats._dis

In [35]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1350674222191923,
              max_delta_step=0, max_depth=10, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=150, n_jobs=-1,
              num_leaves=38, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=0),
 {'learning_rate': 0.1350674222191923, 'num_leaves': 38, 'max_depth': 10},
 0.250552,
 10)

In [36]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape)

(0.7488873349293266, (1800,), (1800, 10))

In [37]:
### HYPERPARAM TUNING WITH HYPEROPT + RECURSIVE FEATURE ADDITION (RFA) ###

model = BoostRFA(
    regr_xgb, param_grid=param_dist_hyperopt, min_features_to_select=1, step=1,
    n_iter=8, sampling_seed=0
)
model.fit(
    X_regr_train, y_regr_train, trials=Trials(), 
    eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0
)


8 trials detected for ('max_depth', 'learning_rate', 'colsample_bytree')

trial: 0001 ### iterations: 00149 ### eval_score: 0.26412
trial: 0002 ### iterations: 00080 ### eval_score: 0.25357
trial: 0003 ### iterations: 00054 ### eval_score: 0.26123
trial: 0004 ### iterations: 00149 ### eval_score: 0.2801
trial: 0005 ### iterations: 00149 ### eval_score: 0.27046
trial: 0006 ### iterations: 00149 ### eval_score: 0.24789
trial: 0007 ### iterations: 00054 ### eval_score: 0.25928
trial: 0008 ### iterations: 00140 ### eval_score: 0.27284


BoostRFA(estimator=XGBRegressor(base_score=None, booster=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, enable_categorical=False,
                                gamma=None, gpu_id=None, importance_type=None,
                                interaction_constraints=None,
                                learning_rate=None, max_delta_step=None,
                                max_depth=None, min_child_weight=None,
                                missing=nan, monotone_constraints=None,
                                n_estimato...
                                random_state=0, reg_alpha=None, reg_lambda=None,
                                scale_pos_weight=None, subsample=None,
                                tree_method=None, validate_parameters=None,
                                verbosity=0),
         min_features_to_select=1, n_iter=8,
         param_grid={'colsample_bytree': <hyperopt.pyll

In [38]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7597292534356749,
              enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.059836658149176665,
              max_delta_step=0, max_depth=16, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=150, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=0),
 {'colsample_bytree': 0.7597292534356749,
  'learning_rate': 0.059836658149176665,
  'max_depth': 16},
 0.247887,
 8)

In [39]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape)

(0.7542006308661441, (1800,), (1800, 8))

# Hyperparameters Tuning + Features Selection with SHAP

In [40]:
### HYPERPARAM TUNING WITH GRID-SEARCH + BORUTA SHAP ###

model = BoostBoruta(
    clf_xgb, param_grid=param_grid, max_iter=200, perc=100,
    importance_type='shap_importances', train_importance=False
)
model.fit(X_clf_train, y_clf_train, eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0)


8 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00024 ### eval_score: 0.20151
trial: 0002 ### iterations: 00020 ### eval_score: 0.20877
trial: 0003 ### iterations: 00024 ### eval_score: 0.20151
trial: 0004 ### iterations: 00020 ### eval_score: 0.20877
trial: 0005 ### iterations: 00048 ### eval_score: 0.20401
trial: 0006 ### iterations: 00048 ### eval_score: 0.20575
trial: 0007 ### iterations: 00048 ### eval_score: 0.20401
trial: 0008 ### iterations: 00048 ### eval_score: 0.20575


BoostBoruta(estimator=XGBClassifier(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=150, n_jobs=-1,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=0, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                               

In [41]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

(XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.2, max_delta_step=0,
               max_depth=10, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=150, n_jobs=-1,
               num_leaves=25, num_parallel_tree=1, predictor='auto',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
               subsample=1, tree_method='exact', validate_parameters=1,
               verbosity=0),
 {'learning_rate': 0.2, 'num_leaves': 25, 'max_depth': 10},
 0.201509,
 10)

In [42]:
(model.score(X_clf_valid, y_clf_valid), 
 model.predict(X_clf_valid).shape, 
 model.transform(X_clf_valid).shape,
 model.predict_proba(X_clf_valid).shape)

(0.9144444444444444, (1800,), (1800, 10), (1800, 2))

In [43]:
### HYPERPARAM TUNING WITH RANDOM-SEARCH + RECURSIVE FEATURE ELIMINATION (RFE) SHAP ###

model = BoostRFE(
    regr_xgb, param_grid=param_dist, min_features_to_select=1, step=1,
    n_iter=8, sampling_seed=0,
    importance_type='shap_importances', train_importance=False
)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)


8 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00021 ### eval_score: 0.25941
trial: 0002 ### iterations: 00064 ### eval_score: 0.25075
trial: 0003 ### iterations: 00075 ### eval_score: 0.25493
trial: 0004 ### iterations: 00084 ### eval_score: 0.25002
trial: 0005 ### iterations: 00093 ### eval_score: 0.25609
trial: 0006 ### iterations: 00039 ### eval_score: 0.2573
trial: 0007 ### iterations: 00074 ### eval_score: 0.25348
trial: 0008 ### iterations: 00032 ### eval_score: 0.2583


BoostRFE(estimator=XGBRegressor(base_score=None, booster=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, enable_categorical=False,
                                gamma=None, gpu_id=None, importance_type=None,
                                interaction_constraints=None,
                                learning_rate=None, max_delta_step=None,
                                max_depth=None, min_child_weight=None,
                                missing=nan, monotone_constraints=None,
                                n_estimato...
                                tree_method=None, validate_parameters=None,
                                verbosity=0),
         importance_type='shap_importances', min_features_to_select=1, n_iter=8,
         param_grid={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5bba146e90>,
                     'max_depth': [10, 12],
                 

In [44]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1669837381562427,
              max_delta_step=0, max_depth=10, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=150, n_jobs=-1,
              num_leaves=25, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=0),
 {'learning_rate': 0.1669837381562427, 'num_leaves': 25, 'max_depth': 10},
 0.250021,
 11)

In [45]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape)

(0.7499501426259738, (1800,), (1800, 11))

In [46]:
### HYPERPARAM TUNING WITH HYPEROPT + RECURSIVE FEATURE ADDITION (RFA) SHAP ###

model = BoostRFA(
    regr_xgb, param_grid=param_dist_hyperopt, min_features_to_select=1, step=1,
    n_iter=8, sampling_seed=0,
    importance_type='shap_importances', train_importance=False
)
model.fit(
    X_regr_train, y_regr_train, trials=Trials(), 
    eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0
)


8 trials detected for ('max_depth', 'learning_rate', 'colsample_bytree')

trial: 0001 ### iterations: 00149 ### eval_score: 0.25811
trial: 0002 ### iterations: 00078 ### eval_score: 0.25554
trial: 0003 ### iterations: 00059 ### eval_score: 0.26658
trial: 0004 ### iterations: 00149 ### eval_score: 0.27356
trial: 0005 ### iterations: 00149 ### eval_score: 0.26426
trial: 0006 ### iterations: 00149 ### eval_score: 0.25537
trial: 0007 ### iterations: 00052 ### eval_score: 0.26107
trial: 0008 ### iterations: 00137 ### eval_score: 0.27787


BoostRFA(estimator=XGBRegressor(base_score=None, booster=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, enable_categorical=False,
                                gamma=None, gpu_id=None, importance_type=None,
                                interaction_constraints=None,
                                learning_rate=None, max_delta_step=None,
                                max_depth=None, min_child_weight=None,
                                missing=nan, monotone_constraints=None,
                                n_estimato...
                                tree_method=None, validate_parameters=None,
                                verbosity=0),
         importance_type='shap_importances', min_features_to_select=1, n_iter=8,
         param_grid={'colsample_bytree': <hyperopt.pyll.base.Apply object at 0x7f5bc2ca6950>,
                     'learning_rate': <hyperopt.pyll.base.Apply object at 0x7

In [47]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7597292534356749,
              enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.059836658149176665,
              max_delta_step=0, max_depth=16, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=150, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=0),
 {'colsample_bytree': 0.7597292534356749,
  'learning_rate': 0.059836658149176665,
  'max_depth': 16},
 0.255374,
 11)

In [48]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape)

(0.7391290836488575, (1800,), (1800, 11))

# CUSTOM EVAL METRIC SUPPORT

In [49]:
from sklearn.metrics import roc_auc_score

def AUC(y_hat, dtrain):
    y_true = dtrain.get_label()
    return 'auc', roc_auc_score(y_true, y_hat)

In [50]:
model = BoostRFE(
    clf_xgb, 
    param_grid=param_grid, min_features_to_select=1, step=1,
    greater_is_better=True
)
model.fit(
    X_clf_train, y_clf_train, 
    eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0,
    eval_metric=AUC
)


8 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00000 ### eval_score: 0.78597
trial: 0002 ### iterations: 00004 ### eval_score: 0.78037
trial: 0003 ### iterations: 00000 ### eval_score: 0.78597
trial: 0004 ### iterations: 00004 ### eval_score: 0.78037
trial: 0005 ### iterations: 00005 ### eval_score: 0.78619
trial: 0006 ### iterations: 00004 ### eval_score: 0.78173
trial: 0007 ### iterations: 00005 ### eval_score: 0.78619
trial: 0008 ### iterations: 00004 ### eval_score: 0.78173


BoostRFE(estimator=XGBClassifier(base_score=None, booster=None,
                                 colsample_bylevel=None, colsample_bynode=None,
                                 colsample_bytree=None,
                                 enable_categorical=False, gamma=None,
                                 gpu_id=None, importance_type=None,
                                 interaction_constraints=None,
                                 learning_rate=None, max_delta_step=None,
                                 max_depth=None, min_child_weight=None,
                                 missing=nan, monotone_constraints=None,
                                 n_estimators=150, n_jobs=-1,
                                 num_parallel_tree=None, predictor=None,
                                 random_state=0, reg_alpha=None,
                                 reg_lambda=None, scale_pos_weight=None,
                                 subsample=None, tree_method=None,
                                 valida